# ORGANELLE CONTACT ANALYSIS

-------

## OBJECTIVE:
In this notebook, the logic for determining locations of contacts between 2 or more organelles is outlined. Additionally, the logic for quantifying organelle contact composition (how much of each contact is present) and morphology (contact site size and shape) is outlined as well. Here, we begin with a walkthrough of how the workflow completes 2-way contacts, then proceed to more complex n-way contacts. 

## SUMMARY OF WORKFLOW STEPS:
 - Inputs
    - Get and Load image and its segmentations
    - Save segmentations to a dictionary
    - Create overlap regions dictionary
 - Run Analysis 
    - Regionprops
    - Distribution

## IMPORTS:

In [1]:
import warnings
import numpy as np
from typing import Any, List, Union
import pandas as pd
from IPython.display import display
from pathlib import Path
import os, sys
import itertools 

from skimage.measure import regionprops_table, label
from skimage.segmentation import watershed

from infer_subc.core.file_io import read_czi_image, read_tiff_image
from infer_subc.core.img import apply_mask
from infer_subc.utils.batch import list_image_files, find_segmentation_tiff_files
from infer_subc.utils.stats import surface_area_from_props, get_XY_distribution, get_Z_distribution, _assert_uint16_labels
from infer_subc.utils.stats_helpers import inkeys

from infer_subc.constants import (TEST_IMG_N,
                                  NUC_CH ,
                                  LYSO_CH ,
                                  MITO_CH ,
                                  GOLGI_CH ,
                                  PEROX_CH ,
                                  ER_CH ,
                                  LD_CH ,
                                  RESIDUAL_CH )

#For Convexhull Errors
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", RuntimeWarning)

## INPUTS:
Load image and segmentations and save segmentations to a dictionary. Using these segmentations, overlaps between 2 or more segmentations are performed and added to their own dictionary.

### User Inputs

##### Image Path
Here, the user should edit any of the values to correctly access the folder containing the __RAW__ image files. The naming of this file will then be used later to collect the corresponding segmentation files.

In [2]:
test_img_n = 0

data_root_path = Path(os.path.expanduser("~")) / "Documents/Python Scripts/Infer-subc-2D"

in_data_path = data_root_path / "raw/shannon"
seg_path = data_root_path/"out"
seg_suffix = "-"
im_type = ".tiff"

img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

out_data_path = data_root_path / "out"
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [3]:
img_data,meta_dict = read_czi_image(test_img_name)

channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

##### Function Inputs
These are inputs that will be entered into the main function by the user. These should be changed to properly represent the user's image.

In [4]:
org_names = ['LD', 'ER', 'golgi', 'lyso', 'mito', 'perox']
masks_file_name= ['nuc', 'cell']
mask = 'cell'
splitter = "_"
include_contact_dist = True
dist_centering_obj = 'nuc'
dist_center_on = False
dist_keep_center_as_bin = True
dist_num_bins = 5
dist_zernike_degrees=None

### Collecting All Organelle Segmentations and Region/Mask Segmentations
These inputs are NOT meant to be changed. These will be used by the program to collect the segmentations based off of the inputs from above and used furtherdown in the pipeline.

In [5]:
org_segs = [read_tiff_image(find_segmentation_tiff_files(test_img_name, (org_names + masks_file_name), seg_path, seg_suffix)[org]) for org in org_names]
region_segs=[read_tiff_image(find_segmentation_tiff_files(test_img_name, (org_names + masks_file_name), seg_path, seg_suffix)[masks_file_name[0]]), read_tiff_image(find_segmentation_tiff_files(test_img_name, (org_names + masks_file_name), seg_path, seg_suffix)[masks_file_name[1]])]
mask = region_segs[masks_file_name.index(mask)]
center_obj = region_segs[masks_file_name.index(dist_centering_obj)]

### Make Dictionaries
These inputs are NOT meant to be changed and are not user facing.

Here we will convert the input list of __organelle segmentations__ and the input list of __organelle names__ into a ***labeled*** dictionary. 

The ***labeled*** format treats the locations of the organelles in the image as positive integers with each different organelle within the organelle type having a different integer value assigned to it; meanwhile, the space without the organelle is assigned 0s. 

In [6]:
organelle_segs = {}                                                       #Initialize dictionary
for idx, name in enumerate(org_names):                                  #Loop across each organelle name
    if name == 'ER':                                                    #Proceed only for ER
        organelle_segs[name]=(org_segs[idx]>0).astype(np.uint16)          #Ensures ER is labeled only as one object & sets it as key for its object segmentation
    else:                                                               #Proceed for other organelles
        organelle_segs[name]=org_segs[idx]                                #Set the organelle name as the key for the corresponding object segmentation

## 2-Way Contact Workflow
First, we will run through how the workflow determines 2-Way contacts. In the final take of this function, all contacts, including n-way contacts will be analyzed.

### Inputs

#### Choose a 2-Way Contact
Due to this function being designed around only functioning for a single contact site given by a string, __users should input__ the desired 2-Way contact to analyze here. 

The format for the contact site string is as follows:
<br>
`organelle 1` + `splitter` + `organelle 2`

For example, if the desired contact site is between the _endoplasmic reticulum_ and _peroxisomes_, and in the above block of code the _endoplasmic reticulum_ is set equal to _`ER`_, the _splitter_ is listed as _`_`_, and _peroxisomes_ are set equal to _`perox`_ in the above block of code, the contact site string would be as follows:
<br>
`ER_perox`


In [7]:
orgs = "ER_perox"

#### Make 2-Way Contacts 
Here we are creating a contact site between the selected organelles. The method for creating pairwise contacts like this will be expanded upon to make nth order contacts. These inputs are NOT meant to be changed and are not user facing.

In [8]:
site = organelle_segs[orgs.split(splitter)[0]] * organelle_segs[orgs.split(splitter)[1]]

### RUN ANALYSIS:
Here the data from the above inputs are analyzed to find everything we can about the contacts from regionprops measurements to the original organelles and their numbers' involved in the contacts to even the distribution of the contacts.

#### Find Contacts Involved in Higher Order (3-Way) Contacts
Here, higher order contacts of the 2-Way contact sites are found. In the case of 2-way contacts, the ones looked at are 3-Way contacts. These are overlapping regions between the 2-way contact site and a organelle not involved in the original contact. This data is then used to determine which 2-Way contacts are also involved in 3-Way or even higher order contacts. 

To find these higher order contacts, we iterate across each of the organelles we have segmentations for. Then, for each organelle we have segmentations for, if they are not present in the original 2-Way contact, we find the contacts between the 2-Way contact and the organelle and use the image to watershed onto the 2-Way contact image. Using the watershed, we take the area that has been filled and remove it from the Higher Order Contact version of the 2-Way contact's site variable.

In [9]:
HOc = site.copy()
for org, val in organelle_segs.items():
    if (org not in orgs.split(splitter)) and np.any(site*val):
        HOc = HOc*(np.invert(watershed(image=(np.invert(site)),
                                       markers=(site*val),
                                       mask=site,
                                       connectivity=np.ones((3, 3, 3), bool))>0))

#### Isolate Images To Within the Cell
Here, we ensure that we are only taking measurements from within the cellmask. This means applying the mask to both the 2-Way contact image and the version of the 2-Way contact image that only contains non-redundant contact sites.

In [10]:
labels = label(apply_mask(site, mask)).astype("int")    #Isolate to only contact sites found within the cell of interest
para_labels = apply_mask((HOc>0), mask).astype("int") * labels #copy labels found in labels to para_labels

#### Create List of Regionprops Measurements And Run Regionprops
Here, a table is created using the regionprops measurements for the contact site.

In [11]:
##########################################
## CREATE LIST OF REGIONPROPS MEASUREMENTS
##########################################
# start with LABEL
properties = ["label"]

# add position
properties = properties + ["centroid", "bbox"]

# add area
properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 

# add shape measurements
properties = properties + ["extent", "euler_number", "solidity", "axis_major_length", "slice"] # "feret_diameter_max", "axis_minor_length", 

##################
## RUN REGIONPROPS
##################
props = regionprops_table(labels, intensity_image=None, properties=properties, extra_properties=None, spacing=scale)

#### Add Surface Area Columns to Data Table
This function adds the surface area of the contact. Viewing the two involved organelles as circles in a venn-diagram and the contact as the overlapping area, the surface area is the part of each circle's surface that is inside of the other circle.

In [12]:
surface_area_tab = pd.DataFrame(surface_area_from_props(labels, props, scale))

#### Determine Organelles Involved in Contact, Organelle's Labels, and Redundancy
This section of code enables the determination of which organelle label of each organelle involved in the contact is present in the contact. For example, if peroxisome # 6 was involved in a contact with the ER, the contact would be listed as "ER_perox" and the label for the contact would be listed as 1_6. 

Additionally, this section of code also determines whether or not the contact is involved in a higher order contact, and lists it as redundant if it is. This means, that any contact that is of the highest order is listed as non-redundant. This does not remove redundant contacts from analysis--it only provides additional context to the data.

In [13]:
cont_inv = []                                               
involved = orgs.split(splitter)                             
indexes = dict.fromkeys(involved, [])                       
indexes[orgs] = []                                          
redundancy = []
label_a = []
index_ab = []
label_b = []
a = _assert_uint16_labels(organelle_segs[orgs.split(splitter)[0]])
b = _assert_uint16_labels(organelle_segs[orgs.split(splitter)[1]])
for index, lab in enumerate(props["label"]):
    present = para_labels[props["slice"][index]]            #examines contact site to find if it is present in a higher order contact or not
    present = present==lab
    redundant = not np.any(present)
    redundancy.append(redundant)
    volume = labels[props["slice"][index]]
    la = a[props["slice"][index]]
    lb = b[props["slice"][index]]
    volume = volume == lab
    la = la[volume]
    lb = lb[volume]

    all_as = np.unique(la[la>0]).tolist()
    all_bs = np.unique(lb[lb>0]).tolist()
    if len(all_as) != 1:
        print(f"we have an error.  as-> {all_as}")
    if len(all_bs) != 1:
        print(f"we have an error.  bs-> {all_bs}")

    label_a.append(f"{all_as[0]}" )
    label_b.append(f"{all_bs[0]}" )
    index_ab.append(f"{all_as[0]}{splitter}{all_bs[0]}")

#### Combine the Datatables
Here, all of the individual datatables created to examine the contacts data present are combined into one data table for export.

In [14]:
props_table = pd.DataFrame(props)
props_table.drop(columns=['slice', 'label'], inplace=True)
props_table.insert(0, 'label',value=index_ab)
props_table.insert(0, "object", orgs)
props_table.rename(columns={"area": "volume"}, inplace=True)

props_table.insert(11, "surface_area", surface_area_tab)
props_table.insert(13, "SA_to_volume_ratio", props_table["surface_area"].div(props_table["volume"]))

if scale is not None:
    round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
    props_table.insert(loc=2, column="scale", value=f"{round_scale}")
else: 
    props_table.insert(loc=2, column="scale", value=f"{tuple(np.ones(labels.ndim))}") 
props_table["redundant"] = list(map(bool, redundancy))


Here, we can view the results of the analysis performed for the contacts thus far.

In [15]:
display(props_table)

,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,...,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,redundant
0,ER_perox,1_2,"(0.4106, 0.0799, 0.0799)",1.879257,9.864230,51.759533,4,121,645,6,...,651,2.008898,0.068232,29.442013,0.506987,0.361111,1,0.684211,0.923434,True
1,ER_perox,1_3,"(0.4106, 0.0799, 0.0799)",1.642376,11.450870,53.466094,4,142,667,5,...,671,0.999441,0.034116,29.295237,0.402396,0.812500,1,inf,0.432237,True
2,ER_perox,1_1,"(0.4106, 0.0799, 0.0799)",1.642376,23.611014,16.513497,4,293,205,5,...,209,1.089312,0.023619,46.120381,0.355976,0.450000,1,inf,0.508868,True
3,ER_perox,1_4,"(0.4106, 0.0799, 0.0799)",2.015643,33.853921,87.371169,4,420,1091,6,...,1097,2.128817,0.057735,36.872165,0.479528,0.229167,1,0.478261,0.741311,True
4,ER_perox,1_5,"(0.4106, 0.0799, 0.0799)",1.750835,39.502869,39.118218,4,491,486,6,...,494,2.454542,0.139089,17.647274,0.642833,0.473214,1,0.803030,0.836116,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,ER_perox,1_420,"(0.4106, 0.0799, 0.0799)",11.086038,93.458043,81.146205,27,1169,1015,28,...,1016,0.187442,0.002624,71.425050,0.171135,1.000000,1,inf,0.000000,True
668,ER_perox,1_415,"(0.4106, 0.0799, 0.0799)",11.496632,84.104244,77.308749,28,1052,967,29,...,968,0.187442,0.002624,71.425050,0.171135,1.000000,1,inf,0.000000,True
669,ER_perox,1_419,"(0.4106, 0.0799, 0.0799)",11.496632,90.300137,84.783794,28,1129,1060,29,...,1062,0.374885,0.005249,71.425050,0.215617,0.500000,1,inf,0.252815,True
670,ER_perox,1_419,"(0.4106, 0.0799, 0.0799)",11.496632,90.500004,84.823767,28,1132,1061,29,...,1062,0.187442,0.002624,71.425050,0.171135,1.000000,1,inf,0.000000,True


#### Calculate Distribution of Measurements
This section of code performs analysis on the distribution measurements of the contacts. It is entirely optional and whether it is performed or not is determined in the user inputs earlier on in this notebook. 

This section of code will additional combine the distribution measurements into a separate table for export.

In [16]:
dist_tabs = []
if include_contact_dist:
    XY_contact_dist, XY_bins, XY_wedges = get_XY_distribution(mask=mask, 
                                                              obj=site,
                                                              obj_name=orgs,
                                                              centering_obj=center_obj,
                                                              scale=scale,
                                                              center_on=dist_center_on,
                                                              keep_center_as_bin=dist_keep_center_as_bin,
                                                              num_bins=dist_num_bins,
                                                              zernike_degrees=dist_zernike_degrees)
          
    Z_contact_dist = get_Z_distribution(mask=mask,
                                        obj=site,
                                        obj_name=orgs,
                                        center_obj=center_obj,
                                        scale=scale)
    contact_dist_tab = pd.merge(XY_contact_dist, Z_contact_dist, on=["object", "scale"])
    dist_tabs.append(contact_dist_tab)
indexes.clear()
combined_dist_tab = pd.concat(dist_tabs, ignore_index=True)
combined_dist_tab.insert(loc=0,column='image_name',value=test_img_name.stem)

WTF!!  how did we have missing labels?


Here, we can view the distribution metrics created by the above block of code. 

>NOTE: If include_contact_dist is False, the following code will produce an error as there will be no data to display.

In [17]:
display(combined_dist_tab)

,image_name,object,scale,XY_n_bins,XY_bins,XY_mask_vox_cnt_perbin,XY_obj_vox_cnt_perbin,XY_center_vox_cnt_perbin,XY_n_pix_perbin,XY_portion_pix_perbin,...,XY_area_wedges_perbin,Z_n_slices,Z_slices,Z_mask_vox_cnt,Z_obj_vox_cnt,Z_center_vox_cnt,Z_height,Z_mask_volume,Z_obj_volume,Z_center_volume
0,20230727_C2-121_conditioned_well 2_cell 1_50uM...,ER_perox,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[391573, 150534, 182433, 203736, 1795494]","[2289, 1678, 2253, 4150, 6018]","[159863, 0, 0, 0, 0]","[13444.0, 6063.0, 8888.0, 13027.0, 380856.0]","[0.031836846816552127, 0.01435784009586102, 0....",...,"[[8.794735385184001, 8.813909953611, 7.9574458...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[9, 139, 35897, 119787, 165147, 205238, 238961...","[0, 0, 0, 0, 262, 1810, 3662, 2945, 1494, 1013...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 109, 2171, 3992, 6...",16.013166,"[0.023618888246146916, 0.36478060735715795, 94...","[0.0, 0.0, 0.0, 0.0, 0.6875720800544991, 4.750...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## N-Way Contact Workflow
First, we will run through how the workflow determines 2-Way contacts. In the final take of this function, all contacts, including n-way contacts will be analyzed.

### Inputs

#### Choose an N-Way Contact
Due to this function being designed around only functioning for a single contact site given by a string, __users should input__ their desired n-Way contact to analyze here. 

The format for the contact site string is as follows:
<br>
`organelle 1` + `splitter` + `organelle 2` + `splitter` + `organelle 3` + ...

For example, if the desired contact site is between the _endoplasmic reticulum_, _peroxisomes_, and _mitochondria_ and in the earlier input block of code the _endoplasmic reticulum_ is set equal to _`ER`_, the _splitter_ is listed as _`_`_, _peroxisomes_ are set equal to _`perox`_, and _mitochondria_ are set equal to _`mito`_, the contact site string would be as follows:
<br>
`ER_perox_mito`


In [18]:
orgs = "ER_perox_mito"

#### Make N-Way Contacts 
Here we are creating a contact site between the selected organelles. The method for creating pairwise contacts from earlier is expanded upon here. Previously, the method directly selected the organelles involved; however, when analyzing a variable number of contacts it is not possible to directly call upon what you want so instead we use a for loop. These inputs are NOT meant to be changed and are not user facing.

In [19]:
site = np.ones_like(organelle_segs[orgs.split(splitter)[0]])
for org in orgs.split(splitter):
    site = site*(organelle_segs[org]>0)

### RUN ANALYSIS:
Here the data from the above inputs are analyzed to find everything we can about the contacts from regionprops measurements to the original organelles and their numbers' involved in the contacts to even the distribution of the contacts.

#### Find Contacts Involved in Higher Order (n+1-Way) Contacts
Here, higher order contacts of the n-Way contact sites are found. These are overlapping regions between the n-Way contact site and a organelle not involved in the original contact. This data is then used to determine which n-Way contacts are also involved in higher order contacts. 

To find these higher order contacts, we iterate across each of the organelles we have segmentations for. Then, for each organelle we have segmentations for, if they are not present in the original n-Way contact, we find the contacts between the n-Way contact and the organelle and use the image to watershed onto the n-Way contact image. Using the watershed, we take the area that has been filled and remove it from the Higher Order Contact version of the n-Way contact's site variable.

Functionally, nothing has changed between the 2-Way and n-Way contacts variations of this code.

In [20]:
HOc = site.copy()
for org, val in organelle_segs.items():
    if (org not in orgs.split(splitter)) and np.any(site*val):
        HOc = HOc*(np.invert(watershed(image=(np.invert(site)),
                                       markers=(site*val),
                                       mask=site,
                                       connectivity=np.ones((3, 3, 3), bool))>0))

#### Isolate Images To Within the Cell
Here, we ensure that we are only taking measurements from within the cellmask. This means applying the mask to both the n-Way contact image and the version of the n-Way contact image that only contains non-redundant contact sites.

Functionally, nothing has changed between the 2-Way and n-Way contacts variations of this code.

In [21]:
labels = label(apply_mask(site, mask)).astype("int")    #Isolate to only contact sites found within the cell of interest
para_labels = apply_mask((HOc>0), mask).astype("int") * labels #copy labels found in labels to para_labels

#### Create List of Regionprops Measurements And Run Regionprops
Here, a table is created using the regionprops measurements for the contact site.

Functionally, nothing has changed between the 2-Way and n-Way contacts variations of this code.

In [22]:
##########################################
## CREATE LIST OF REGIONPROPS MEASUREMENTS
##########################################
# start with LABEL
properties = ["label"]

# add position
properties = properties + ["centroid", "bbox"]

# add area
properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 

# add shape measurements
properties = properties + ["extent", "euler_number", "solidity", "axis_major_length", "slice"] # "feret_diameter_max", "axis_minor_length", 

##################
## RUN REGIONPROPS
##################
props = regionprops_table(labels, intensity_image=None, properties=properties, extra_properties=None, spacing=scale)

#### Add Surface Area Columns to Data Table
This function adds the surface area of the contact. Viewing the involved organelles as circles in a venn-diagram and the contact as the overlapping area, the surface area is the part of each circle's surface that is inside of the other circle.

Functionally, nothing has changed between the 2-Way and n-Way contacts variations of this code.

In [23]:
surface_area_tab = pd.DataFrame(surface_area_from_props(labels, props, scale))

#### Determine Organelles Involved in Contact and Organelle's Labels
This section of code enables the determination of which organelle label of each organelle involved in the contact is present in the contact. For example, if peroxisome # 6 was involved in a contact with the ER and mitochondria #43, the contact would be listed as "ER_perox_mito" and the label for the contact would be listed as 1_6_43. 

This section of code also determines whether or not the contact is involved in a higher order contact, and lists it as redundant if it is. This means, that any contact that is of the highest order is listed as non-redundant. This does not remove redundant contacts from analysis--it only provides additional context to the data.

Furthermore, this section of code has been overhauled here to enable iterating across n organelles rather than just 2-Way contacts. This enables for 3, 4, 5, 6, 7, and so on contacts between organelles to be measured.

In [24]:
cont_inv = []                                               #initializes a variable to be used for creating the values for the contacts in the dictionary
                                                                  #   This variable is a list of the labels of each organelle involved in one contact site between those organelles
involved = orgs.split(splitter)                             #creates list of all involved organelles in the contact
indexes = dict.fromkeys(involved, [])                       #A dictionary of indexes of site involved in the contact
indexes[orgs] = []                                           #   str = "contact" or "organelle", may have multiple different organelles
                                                                  #   list = contact or organelle number in image corresponding to the same contact in the other keys
                                                                #   a 2-way contact will have 3 keys, a 3-way contact will have 4 keys, etc
redundancy = []
for index, l in enumerate(props["label"]):
    cont_inv.clear()                                        #clears cont_inv variable of any labels from past contact site for new contact site
    present = para_labels[props["slice"][index]]            #examines contact site to find if it is present in a higher order contact or not
    present = present==l
    redundant = not np.any(present)
    redundancy.append(redundant)
    for org in involved:                                    #iterates across list of involved organelles
        volume = labels[props["slice"][index]]
        lorg = organelle_segs[org][props["slice"][index]]
        volume = volume==l
        lorg = lorg[volume]
        all_inv = np.unique(lorg[lorg>0]).tolist()
        if len(all_inv) != 1:                               #ensures only one label is involved in the contact site
            print(f"we have an error.  as-> {all_inv}")     #informs the console of any errors and the reasoing for it
        indexes[org].append(f"{all_inv[0]}")                #adds the label of the organelle involved in the contact to the organelle's key's list
        cont_inv.append(f"{all_inv[0]}")                    #adds the label of the organelle involved in the contact to the list of involved organelle labels
    indexes[orgs].append("_".join(cont_inv))                #adds the combination of all the organelle's labels involved in the contact to the contact key's list

#### Combine the Datatables
Here, all of the individual datatables created to examine the contacts data present are combined into one data table for export.

In [25]:
props_table = pd.DataFrame(props)
props_table.drop(columns=['slice', 'label'], inplace=True)
props_table.insert(0, 'label',value=indexes[orgs])
props_table.insert(0, "object", orgs)
props_table.rename(columns={"area": "volume"}, inplace=True)
props_table.insert(11, "surface_area", surface_area_tab)
props_table.insert(13, "SA_to_volume_ratio", 
props_table["surface_area"].div(props_table["volume"]))
if scale is not None:
    round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
    props_table.insert(loc=2, column="scale", value=f"{round_scale}")
else: 
    props_table.insert(loc=2, column="scale", value=f"{tuple(np.ones(labels.ndim))}") 
props_table["redundant"] = list(map(bool, redundancy))

Here, we can view the results of the analysis performed for the contacts thus far.

In [26]:
display(props_table)

,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,bbox-3,...,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,redundant
0,ER_perox_mito,1_1_2,"(0.4106, 0.0799, 0.0799)",1.642376,23.477769,16.495731,4,293,206,5,...,208,0.421324,0.007873,53.515276,0.246820,0.75,1,inf,0.206422,False
1,ER_perox_mito,1_5_21,"(0.4106, 0.0799, 0.0799)",1.642376,39.293951,39.293951,4,491,491,5,...,493,0.374885,0.005249,71.425050,0.215617,0.50,1,inf,0.252815,False
2,ER_perox_mito,1_5_21,"(0.4106, 0.0799, 0.0799)",1.642376,39.693686,38.974163,4,496,487,5,...,489,0.374885,0.005249,71.425050,0.215617,0.50,1,inf,0.252815,False
3,ER_perox_mito,1_6_24,"(0.4106, 0.0799, 0.0799)",1.642376,40.453182,39.333924,4,506,492,5,...,493,0.187442,0.002624,71.425050,0.171135,1.00,1,inf,0.000000,False
4,ER_perox_mito,1_8_29,"(0.4106, 0.0799, 0.0799)",1.916105,45.862929,62.358660,4,573,780,6,...,781,0.473719,0.007873,60.170340,0.246820,0.75,1,inf,0.869819,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,ER_perox_mito,1_228_742,"(0.4106, 0.0799, 0.0799)",8.211880,89.940375,74.350710,20,1125,930,21,...,931,0.187442,0.002624,71.425050,0.171135,1.00,1,inf,0.000000,False
256,ER_perox_mito,1_366_747,"(0.4106, 0.0799, 0.0799)",8.622474,82.265463,73.871028,21,1029,924,22,...,925,0.187442,0.002624,71.425050,0.171135,1.00,1,inf,0.000000,False
257,ER_perox_mito,1_392_619,"(0.4106, 0.0799, 0.0799)",8.622474,88.861091,75.749783,21,1111,947,22,...,949,0.374885,0.005249,71.425050,0.215617,0.50,1,inf,0.252815,False
258,ER_perox_mito,1_398_735,"(0.4106, 0.0799, 0.0799)",9.033068,82.825092,74.110869,22,1036,927,23,...,928,0.187442,0.002624,71.425050,0.171135,1.00,1,inf,0.000000,False


#### Calculate Distribution of Measurements
This section of code performs analysis on the distribution measurements of the contacts. It is entirely optional and whether it is performed or not is determined in the user inputs earlier on in this notebook. 

This section of code will additional combine the distribution measurements into a separate table for export.

In [27]:
dist_tabs = []
if include_contact_dist:
    XY_contact_dist, XY_bins, XY_wedges = get_XY_distribution(mask=mask, 
                                                              obj=site,
                                                              obj_name=orgs,
                                                              centering_obj=center_obj,
                                                              scale=scale,
                                                              center_on=dist_center_on,
                                                              keep_center_as_bin=dist_keep_center_as_bin,
                                                              num_bins=dist_num_bins,
                                                              zernike_degrees=dist_zernike_degrees)
          
    Z_contact_dist = get_Z_distribution(mask=mask,
                                        obj=site,
                                        obj_name=orgs,
                                        center_obj=center_obj,
                                        scale=scale)
    contact_dist_tab = pd.merge(XY_contact_dist, Z_contact_dist, on=["object", "scale"])
    dist_tabs.append(contact_dist_tab)
indexes.clear()
combined_dist_tab = pd.concat(dist_tabs, ignore_index=True)
combined_dist_tab.insert(loc=0,column='image_name',value=test_img_name.stem)

WTF!!  how did we have missing labels?


Here, we can view the distribution metrics created by the above block of code. 

>NOTE: If include_contact_dist is False, the following code will produce an error as there will be no data to display.

In [28]:
display(combined_dist_tab)

,image_name,object,scale,XY_n_bins,XY_bins,XY_mask_vox_cnt_perbin,XY_obj_vox_cnt_perbin,XY_center_vox_cnt_perbin,XY_n_pix_perbin,XY_portion_pix_perbin,...,XY_area_wedges_perbin,Z_n_slices,Z_slices,Z_mask_vox_cnt,Z_obj_vox_cnt,Z_center_vox_cnt,Z_height,Z_mask_volume,Z_obj_volume,Z_center_volume
0,20230727_C2-121_conditioned_well 2_cell 1_50uM...,ER_perox_mito,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[391573, 150534, 182433, 203736, 1795494]","[216, 103, 122, 184, 492]","[159863, 0, 0, 0, 0]","[13444.0, 6063.0, 8888.0, 13027.0, 380856.0]","[0.031836846816552127, 0.01435784009586102, 0....",...,"[[8.794735385184001, 8.813909953611, 7.9574458...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[9, 139, 35897, 119787, 165147, 205238, 238961...","[0, 0, 0, 0, 18, 136, 286, 272, 101, 43, 67, 4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 109, 2171, 3992, 6...",16.013166,"[0.023618888246146916, 0.36478060735715795, 94...","[0.0, 0.0, 0.0, 0.0, 0.04723777649229383, 0.35...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## DEFINE FUNCTIONS:
This section creates new functions that should run the contact metrics. These functions are based on the code from above.

### Make Dictionary
Creates a dictionary of all organelle segmentations with their keys assigned to their names

In [29]:
def _make_dict(obj_names: list[str],                                        #Intakes list of object names
               obj_segs: list[np.ndarray]):                                 #Intakes list of object segmentations
    objs_labeled = {}                                                       #Initialize dictionary
    for idx, name in enumerate(obj_names):                                  #Loop across each organelle name
        if name == 'ER':                                                    #Proceed only for ER
            objs_labeled[name]=(obj_segs[idx]>0).astype(np.uint16)          #Ensures ER is labeled only as one object & sets it as key for its object segmentation
        else:                                                               #Proceed for other organelles
            objs_labeled[name]=obj_segs[idx]                                #Set the organelle name as the key for the corresponding object segmentation
    return objs_labeled                                                 #Return a dictionary of segmented objects with keys as the organelle name


### Contact
Creates a contact between two or more organelles and finds if there are any higher order contacts present for the contact sites between those organelles.

In [30]:
def _contact(orgs:str,
             organelle_segs: dict[str:np.ndarray]) -> tuple[np.ndarray, np.ndarray]: 
    ##########################################
    ## CREATE CONTACT
    ##########################################
    site = np.ones_like(organelle_segs[orgs.split(splitter)[0]])
    for org in orgs.split(splitter):
        site = site*(organelle_segs[org]>0)
        
    ##########################################
    ## DETERMINE REDUNDANT CONTACTS
    ##########################################
    HOc = site.copy()
    for org, val in organelle_segs.items():
        if (org not in orgs.split(splitter)) and np.any(site*val):
            HOc = HOc*(np.invert(watershed(image=(np.invert(site)),
                                           markers=(site*val),
                                            mask=site,
                                           connectivity=np.ones((3, 3, 3), bool))>0))
    return site, HOc

### Get Contact Metrics
This function combines multiple steps seen in the above "Get Contact Metrics for Chosen n-Way Contact" section to produce a data table for the contact metrics and provides a True or False statement to enable or disable the production of a data table for the distribution metrics.

In [33]:
def _get_contact_metrics_3D(orgs: str,
                        organelle_segs: dict[str:np.ndarray],
                        mask: np.ndarray,
                        splitter: str="_",
                        scale: Union[tuple, None]=None,
                        include_dist:bool=False, 
                        dist_centering_obj: Union[np.ndarray, None]=None,
                        dist_num_bins: Union[int, None]=None,
                        dist_zernike_degrees: Union[int, None]=None,
                        dist_center_on: Union[bool, None]=None,
                        dist_keep_center_as_bin: Union[bool, None]=None) -> list:
    
    ##########################################
    ## PREPARING CONTACT METRICS
    ##########################################
    site, HOc = _contact(orgs, organelle_segs)    
    dist_tabs =[]
    labels = label(apply_mask(site, mask)).astype("int")    #Isolate to only contact sites found within the cell of interest
    para_labels = apply_mask((HOc>0), mask).astype("int") * labels #copy labels found in labels to para_labels
    
    ##########################################
    ## CREATE LIST OF REGIONPROPS MEASUREMENTS
    ##########################################
    # start with LABEL
    properties = ["label"]

    # add position
    properties = properties + ["centroid", "bbox"]

    # add area
    properties = properties + ["area", "equivalent_diameter"] # "num_pixels", 

    # add shape measurements
    properties = properties + ["extent", "euler_number", "solidity", "axis_major_length", "slice"] # "feret_diameter_max", "axis_minor_length", 

    ##################
    ## RUN REGIONPROPS
    ##################
    props = regionprops_table(labels, intensity_image=None, properties=properties, extra_properties=None, spacing=scale)

    ##################################################################
    ## RUN SURFACE AREA FUNCTION SEPARATELY AND APPEND THE PROPS_TABLE
    ##################################################################
    surface_area_tab = pd.DataFrame(surface_area_from_props(labels, props, scale))

    ######################################################
    ## LIST WHICH ORGANELLES ARE INVOLVED IN THE CONTACT
    ######################################################    
    cont_inv = []                                               #initializes a variable to be used for creating the values for the contacts in the dictionary
                                                                  #   This variable is a list of the labels of each organelle involved in one contact site between those organelles
    involved = orgs.split(splitter)                             #creates list of all involved organelles in the contact
    indexes = dict.fromkeys(involved, [])                       #A dictionary of indexes of site involved in the contact
    indexes[orgs] = []                                           #   str = "contact" or "organelle", may have multiple different organelles
                                                                  #   list = contact or organelle number in image corresponding to the same contact in the other keys
                                                                #   a 2-way contact will have 3 keys, a 3-way contact will have 4 keys, etc
    redundancy = []
    for index, l in enumerate(props["label"]):
        cont_inv.clear()                                        #clears cont_inv variable of any labels from past contact site for new contact site
        present = para_labels[props["slice"][index]]
        present = present==l
        redundant = not np.any(present)
        redundancy.append(redundant)
        for org in involved:                                    #iterates across list of involved organelles
            volume = labels[props["slice"][index]]
            lorg = organelle_segs[org][props["slice"][index]]   
            volume = volume==l                                  
            lorg = lorg[volume]                                 
            all_inv = np.unique(lorg[lorg>0]).tolist()          
            if len(all_inv) != 1:                               #ensures only one label is involved in the contact site
                print(f"we have an error.  as-> {all_inv}")     #informs the console of any errors and the reasoing for it
            indexes[org].append(f"{all_inv[0]}")                #adds the label of the organelle involved in the contact to the organelle's key's list
            cont_inv.append(f"{all_inv[0]}")                    #adds the label of the organelle involved in the contact to the list of involved organelle labels
        indexes[orgs].append(splitter.join(cont_inv))           #adds the combination of all the organelle's labels involved in the contact to the contact key's list                                                
    ######################################################
    ## CREATE COMBINED DATAFRAME OF THE QUANTIFICATION
    ######################################################
    props_table = pd.DataFrame(props)
    props_table.drop(columns=['slice', 'label'], inplace=True)
    props_table.insert(0, 'label',value=indexes[orgs])
    props_table.insert(0, "object", orgs)
    props_table.rename(columns={"area": "volume"}, inplace=True)

    props_table.insert(11, "surface_area", surface_area_tab)
    props_table.insert(13, "SA_to_volume_ratio", 
    props_table["surface_area"].div(props_table["volume"]))
    props_table["redundant"] = list(map(bool, redundancy))
    
    if scale is not None:
        round_scale = (round(scale[0], 4), round(scale[1], 4), round(scale[2], 4))
        props_table.insert(loc=2, column="scale", value=f"{round_scale}")
    else: 
        props_table.insert(loc=2, column="scale", value=f"{tuple(np.ones(labels.ndim))}") 

    ######################################################
    ## optional: DISTRIBUTION OF CONTACTS MEASUREMENTS
    ######################################################
    if include_dist:
        XY_contact_dist, XY_bins, XY_wedges = get_XY_distribution(mask=mask, 
                                                                  obj=site,
                                                                  obj_name=orgs,
                                                                  centering_obj=dist_centering_obj,
                                                                  scale=scale,
                                                                  center_on=dist_center_on,
                                                                  keep_center_as_bin=dist_keep_center_as_bin,
                                                                  num_bins=dist_num_bins,
                                                                  zernike_degrees=dist_zernike_degrees)
       
        Z_contact_dist = get_Z_distribution(mask=mask,
                                            obj=site,
                                            obj_name=orgs,
                                            center_obj=dist_centering_obj,
                                            scale=scale)
        contact_dist_tab = pd.merge(XY_contact_dist, Z_contact_dist, on=["object", "scale"])
        dist_tabs.append(contact_dist_tab)
    indexes.clear()
    if include_dist:
        return props_table, dist_tabs
    else:
        return props_table

## OUTPUT:
Here, we will use the functions we created above to create the output from earlier that was made without using the functions while expanding on it to repeat across all of the contact sites.

In [34]:
distance_tabs = []
contacts_tabs = []
all_pos =[]
labeled_dict = _make_dict(org_names, org_segs)
for n in list(map(lambda x:x+2, (range(len(org_names)-1)))):
        all_pos += itertools.combinations(org_names, n)
possib = [splitter.join(cont) for cont in all_pos]
if include_contact_dist:
    for conts in possib:
        print(conts)
        cont_tab, dist_tab = _get_contact_metrics_3D(orgs=conts,
                                                    organelle_segs=labeled_dict,
                                                    mask=mask,
                                                    splitter=splitter,
                                                    scale=scale,
                                                    include_dist=include_contact_dist,
                                                    dist_centering_obj=region_segs[masks_file_name.index(dist_centering_obj)],
                                                    dist_num_bins=dist_num_bins,
                                                    dist_zernike_degrees=dist_zernike_degrees,
                                                    dist_center_on=dist_center_on,
                                                    dist_keep_center_as_bin=dist_keep_center_as_bin)
        for tabs in dist_tab:
            distance_tabs.append(tabs)
        contacts_tabs.append(cont_tab)
else:
    for conts in all_pos:
        cont_tab = _get_contact_metrics_3D(orgs=orgs,
                                           organelle_segs=labeled_dict,
                                           mask=mask,
                                           splitter=splitter,
                                           scale=scale,
                                           include_dist=include_contact_dist)
        contacts_tabs.append(cont_tab)

LD_ER
WTF!!  how did we have missing labels?
LD_golgi
WTF!!  how did we have missing labels?
LD_lyso
WTF!!  how did we have missing labels?
LD_mito
WTF!!  how did we have missing labels?
LD_perox
WTF!!  how did we have missing labels?
ER_golgi
WTF!!  how did we have missing labels?
ER_lyso
WTF!!  how did we have missing labels?
ER_mito
WTF!!  how did we have missing labels?
ER_perox
WTF!!  how did we have missing labels?
golgi_lyso
WTF!!  how did we have missing labels?
golgi_mito
WTF!!  how did we have missing labels?
golgi_perox
WTF!!  how did we have missing labels?
lyso_mito
WTF!!  how did we have missing labels?
lyso_perox
WTF!!  how did we have missing labels?
mito_perox
WTF!!  how did we have missing labels?
LD_ER_golgi
WTF!!  how did we have missing labels?
LD_ER_lyso
WTF!!  how did we have missing labels?
LD_ER_mito
WTF!!  how did we have missing labels?
LD_ER_perox
WTF!!  how did we have missing labels?
LD_golgi_lyso
WTF!!  how did we have missing labels?
LD_golgi_mito
WTF!! 

The following block of code enables viewing of the contact metrics table.

In [37]:
final_contact_tab = pd.concat(contacts_tabs, ignore_index=True)
final_contact_tab['redundant'] = final_contact_tab['redundant'].astype(bool)
final_contact_tab.insert(loc=0,column='image_name',value=test_img_name.stem)
display(final_contact_tab)

,image_name,object,label,scale,centroid-0,centroid-1,centroid-2,bbox-0,bbox-1,bbox-2,...,bbox-5,surface_area,volume,SA_to_volume_ratio,equivalent_diameter,extent,euler_number,solidity,axis_major_length,redundant
0,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_ER,3_1,"(0.4106, 0.0799, 0.0799)",1.642376,40.213341,40.350393,4,502,504,...,507,0.680771,0.018370,37.058370,0.327370,0.777778,1,inf,0.270270,False
1,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_ER,4_1,"(0.4106, 0.0799, 0.0799)",1.642376,44.477181,60.200091,4,556,752,...,755,0.508653,0.007873,64.607620,0.246820,0.500000,1,inf,0.328671,False
2,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_ER,5_1,"(0.4106, 0.0799, 0.0799)",1.642376,59.000886,63.158130,4,738,790,...,791,0.187442,0.002624,71.425050,0.171135,1.000000,1,inf,0.000000,True
3,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_ER,7_1,"(0.4106, 0.0799, 0.0799)",2.015643,74.779517,68.689009,4,934,858,...,861,0.973331,0.028868,33.717169,0.380601,0.458333,1,0.916667,0.563715,True
4,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_ER,11_1,"(0.4106, 0.0799, 0.0799)",2.052970,54.534514,62.236075,5,680,777,...,781,1.078681,0.039365,27.402157,0.422056,0.750000,1,inf,0.412845,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4755,20230727_C2-121_conditioned_well 2_cell 1_50uM...,ER_lyso_mito_perox,1_34_220_24,"(0.4106, 0.0799, 0.0799)",2.052970,74.270763,69.154155,5,928,865,...,866,0.454980,0.007873,57.790189,0.246820,1.000000,1,inf,0.291925,False
4756,20230727_C2-121_conditioned_well 2_cell 1_50uM...,ER_lyso_mito_perox,1_39_144_83,"(0.4106, 0.0799, 0.0799)",2.052970,91.579289,78.907689,5,1145,987,...,988,0.321211,0.005249,61.198904,0.215617,1.000000,1,inf,0.178767,False
4757,20230727_C2-121_conditioned_well 2_cell 1_50uM...,ER_lyso_mito_perox,1_61_28_104,"(0.4106, 0.0799, 0.0799)",2.463564,50.526504,58.920939,6,632,737,...,738,0.187442,0.002624,71.425050,0.171135,1.000000,1,inf,0.000000,False
4758,20230727_C2-121_conditioned_well 2_cell 1_50uM...,ER_lyso_mito_perox,1_14_222_112,"(0.4106, 0.0799, 0.0799)",2.463564,84.264138,74.670498,6,1054,934,...,935,0.187442,0.002624,71.425050,0.171135,1.000000,1,inf,0.000000,False


The following block of code enables viewing of the distribution metrics table.

In [38]:
final_dist_tab = pd.concat(distance_tabs, ignore_index=True)
final_dist_tab.insert(loc=0,column='image_name',value=test_img_name.stem)
display(final_dist_tab)

,image_name,object,scale,XY_n_bins,XY_bins,XY_mask_vox_cnt_perbin,XY_obj_vox_cnt_perbin,XY_center_vox_cnt_perbin,XY_n_pix_perbin,XY_portion_pix_perbin,...,XY_area_wedges_perbin,Z_n_slices,Z_slices,Z_mask_vox_cnt,Z_obj_vox_cnt,Z_center_vox_cnt,Z_height,Z_mask_volume,Z_obj_volume,Z_center_volume
0,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_ER,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[391573, 150534, 182433, 203736, 1795494]","[0, 18, 12, 107, 39]","[159863, 0, 0, 0, 0]","[13444.0, 6063.0, 8888.0, 13027.0, 380856.0]","[0.031836846816552127, 0.01435784009586102, 0....",...,"[[8.794735385184001, 8.813909953611, 7.9574458...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[9, 139, 35897, 119787, 165147, 205238, 238961...","[0, 0, 0, 0, 12, 25, 89, 11, 0, 18, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 109, 2171, 3992, 6...",16.013166,"[0.023618888246146916, 0.36478060735715795, 94...","[0.0, 0.0, 0.0, 0.0, 0.031491850994862555, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_golgi,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[391573, 150534, 182433, 203736, 1795494]","[0, 0, 0, 0, 0]","[159863, 0, 0, 0, 0]","[13444.0, 6063.0, 8888.0, 13027.0, 380856.0]","[0.031836846816552127, 0.01435784009586102, 0....",...,"[[8.794735385184001, 8.813909953611, 7.9574458...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[9, 139, 35897, 119787, 165147, 205238, 238961...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 109, 2171, 3992, 6...",16.013166,"[0.023618888246146916, 0.36478060735715795, 94...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_lyso,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[391573, 150534, 182433, 203736, 1795494]","[0, 0, 0, 5, 0]","[159863, 0, 0, 0, 0]","[13444.0, 6063.0, 8888.0, 13027.0, 380856.0]","[0.031836846816552127, 0.01435784009586102, 0....",...,"[[8.794735385184001, 8.813909953611, 7.9574458...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[9, 139, 35897, 119787, 165147, 205238, 238961...","[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 109, 2171, 3992, 6...",16.013166,"[0.023618888246146916, 0.36478060735715795, 94...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0131216045811...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_mito,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[391573, 150534, 182433, 203736, 1795494]","[0, 0, 0, 1, 6]","[159863, 0, 0, 0, 0]","[13444.0, 6063.0, 8888.0, 13027.0, 380856.0]","[0.031836846816552127, 0.01435784009586102, 0....",...,"[[8.794735385184001, 8.813909953611, 7.9574458...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[9, 139, 35897, 119787, 165147, 205238, 238961...","[0, 0, 0, 0, 6, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 109, 2171, 3992, 6...",16.013166,"[0.023618888246146916, 0.36478060735715795, 94...","[0.0, 0.0, 0.0, 0.0, 0.015745925497431278, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,20230727_C2-121_conditioned_well 2_cell 1_50uM...,LD_perox,"(0.4106, 0.0799, 0.0799)",5,"[1, 2, 3, 4, 5]","[391573, 150534, 182433, 203736, 1795494]","[0, 0, 0, 72, 3]","[159863, 0, 0, 0, 0]","[13444.0, 6063.0, 8888.0, 13027.0, 380856.0]","[0.031836846816552127, 0.01435784009586102, 0....",...,"[[8.794735385184001, 8.813909953611, 7.9574458...",39,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[9, 139, 35897, 119787, 165147, 205238, 238961...","[0, 0, 0, 0, 0, 3, 66, 6, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 109, 2171, 3992, 6...",16.013166,"[0.023618888246146916, 0.36478060735715795, 94...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.007872962748715639...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,20230727_C2-121_conditioned_well 2_cell 1_50uM...,ER_golgi,"(0.4106, 0.0799, 